## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-29-14-45-05 +0000,nypost,National Guard on ‘alert’ with roving patrols ...,https://nypost.com/2025/11/29/us-news/national...
1,2025-11-29-14-44-01 +0000,nypost,FIFA World Cup soccer final could be most expe...,https://nypost.com/2025/11/29/world-news/fifa-...
2,2025-11-29-14-42-33 +0000,nypost,Effort underway to restore Tappan Zee name to ...,https://nypost.com/2025/11/29/us-news/petition...
3,2025-11-29-14-38-46 +0000,nypost,Over-the-top: Greenwich Village couple goes vi...,https://nypost.com/2025/11/29/us-news/greenwic...
4,2025-11-29-14-37-41 +0000,nypost,NORAD’s Santa tracker kicks off holiday season...,https://nypost.com/2025/11/29/world-news/norad...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
80,trump,38
178,shooting,15
1,guard,14
0,national,12
216,president,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
156,2025-11-28-21-31-00 +0000,wsj,President Trump said he was revoking all execu...,https://www.wsj.com/politics/trump-says-he-is-...,104
89,2025-11-29-06-53-08 +0000,nypost,Trump says he plans to pardon former Honduran ...,https://nypost.com/2025/11/29/us-news/trump-pl...,92
70,2025-11-29-09-41-59 +0000,nypost,Rep. Wasserman Schultz says Trump should blame...,https://nypost.com/2025/11/29/us-news/wasserma...,84
226,2025-11-28-15-48-03 +0000,nyt,Trump Uses National Guard Shooting to Cast Sus...,https://www.nytimes.com/2025/11/27/us/politics...,83
128,2025-11-29-00-35-54 +0000,wapo,Trump plans to pardon former Honduran presiden...,https://www.washingtonpost.com/world/2025/11/2...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
156,104,2025-11-28-21-31-00 +0000,wsj,President Trump said he was revoking all execu...,https://www.wsj.com/politics/trump-says-he-is-...
231,61,2025-11-28-15-17-05 +0000,bbc,National Guard member dies after shooting in W...,https://www.bbc.com/news/articles/cwyx4y6007lo...
221,60,2025-11-28-16-08-10 +0000,nypost,Zelensky’s top aide Andriy Yermak resigns afte...,https://nypost.com/2025/11/28/world-news/zelen...
186,51,2025-11-28-18-45-45 +0000,nypost,Dog saves retiree from narrowly being killed i...,https://nypost.com/2025/11/28/world-news/elder...
135,48,2025-11-28-23-50-12 +0000,nypost,Treasury Secretary Scott Bessent moves to ‘cut...,https://nypost.com/2025/11/28/us-news/treasury...
162,36,2025-11-28-20-46-46 +0000,latimes,Will California's new K-12 antisemitism law ma...,https://www.latimes.com/california/story/2025-...
92,35,2025-11-29-05-32-13 +0000,nyt,"Trump to Pardon Juan Orlando Hernández, Hondur...",https://www.nytimes.com/2025/11/28/world/ameri...
184,35,2025-11-28-18-50-28 +0000,latimes,Black Friday shoppers spend more time looking ...,https://www.latimes.com/business/story/2025-11...
192,31,2025-11-28-18-15-31 +0000,nyt,Israeli Forces Kill Two Palestinians in West B...,https://www.nytimes.com/2025/11/28/world/middl...
15,24,2025-11-29-13-50-50 +0000,nypost,Russia attack on Kyiv kills three as efforts t...,https://nypost.com/2025/11/29/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
